In [19]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import html, re

def sanitize_input(text: str) -> str:
    """SPARQLインジェクション対策の簡易サニタイズ"""
    text = re.sub(r'["\'`]', '', text)
    return html.escape(text)

def search_ndl_by_author(author_name, limit=10):
    sparql = SPARQLWrapper("https://jpsearch.go.jp/rdf/sparql")
    sparql.setTimeout(5)  # タイムアウトを5秒に設定

    query = f"""
        PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema:<http://schema.org/>
        SELECT ?item ?title ?creator ?date ?image ?description ?type WHERE {{
          ?item schema:name     ?title ;
                schema:creator  ?creatorURI ;
                a ?type .
          ?creatorURI rdfs:label ?creator .
          OPTIONAL {{ ?item schema:datePublished ?date }}
          OPTIONAL {{ ?item schema:image ?image }}
          OPTIONAL {{ ?item schema:description ?description }}
          FILTER(CONTAINS(STR(?creator), "{sanitize_input(author_name)}"))
        }} LIMIT {limit}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.query().convert()["results"]["bindings"]
    except Exception as e:
        print(f"接続エラー: {e}")
        return pd.DataFrame([{
            "タイトル": "接続エラーが発生しました",
            "著者": author_name,
            "出版年": "",
            "画像URL": "",
            "説明": str(e),
            "資料タイプ": "",
            "リンク": ""
        }])

    if not results:
        print("該当する著者の資料は見つかりませんでした。")
        return pd.DataFrame([{
            "タイトル": "該当なし",
            "著者": author_name,
            "出版年": "",
            "画像URL": "",
            "説明": "",
            "資料タイプ": "",
            "リンク": ""
        }])

    return pd.DataFrame([{
        "タイトル": r["title"]["value"],
        "著者": r["creator"]["value"],
        "出版年": r.get("date", {}).get("value", ""),
        "画像URL": r.get("image", {}).get("value", ""),
        "説明": r.get("description", {}).get("value", ""),
        "資料タイプ": r.get("type", {}).get("value", ""),
        "リンク": r["item"]["value"]
    } for r in results])

def search_ndl_cross_by_keyword(keyword, limit=20):
    sparql = SPARQLWrapper("https://jpsearch.go.jp/rdf/sparql")
    sparql.setTimeout(5)  # タイムアウトを5秒に設定
    sanitized = sanitize_input(keyword)

    query = f"""
        PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX schema:<http://schema.org/>
        SELECT ?item ?title ?creator ?date ?image ?description ?type WHERE {{
          ?item schema:name ?title .
          OPTIONAL {{ ?item schema:creator ?creatorURI .
                      ?creatorURI rdfs:label ?creator }}
          OPTIONAL {{ ?item schema:datePublished ?date }}
          OPTIONAL {{ ?item schema:image ?image }}
          OPTIONAL {{ ?item schema:description ?description }}
          OPTIONAL {{ ?item a ?type }}
          FILTER (
            CONTAINS(LCASE(STR(?title)), LCASE("{sanitized}")) ||
            CONTAINS(LCASE(STR(?description)), LCASE("{sanitized}")) ||
            CONTAINS(LCASE(STR(?creator)), LCASE("{sanitized}"))
          )
        }} LIMIT {limit}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    try:
        results = sparql.query().convert()["results"]["bindings"]
    except Exception as e:
        print(f"接続エラー: {e}")
        return pd.DataFrame([{
            "タイトル": "接続エラーが発生しました",
            "著者": "",
            "出版年": "",
            "画像URL": "",
            "説明": str(e),
            "資料タイプ": "",
            "リンク": ""
        }])

    if not results:
        print("該当するキーワードの資料は見つかりませんでした。")
        return pd.DataFrame([{
            "タイトル": "該当なし",
            "著者": "",
            "出版年": "",
            "画像URL": "",
            "説明": "",
            "資料タイプ": "",
            "リンク": ""
        }])

    return pd.DataFrame([{
        "タイトル": r["title"]["value"],
        "著者": r.get("creator", {}).get("value", "不明"),
        "出版年": r.get("date", {}).get("value", ""),
        "画像URL": r.get("image", {}).get("value", ""),
        "説明": r.get("description", {}).get("value", ""),
        "資料タイプ": r.get("type", {}).get("value", ""),
        "リンク": r["item"]["value"]
    } for r in results])


In [21]:
df_author = search_ndl_by_author("生成AI")
df_author


接続エラー: The read operation timed out


,タイトル,著者,出版年,画像URL,説明,資料タイプ,リンク
0,接続エラーが発生しました,生成AI,,,The read operation timed out,,
